In [1]:
import os
from pathlib import Path
import torch, numpy as np, pandas as pd
from fastai.vision.all import *

Download and map!

In [2]:
df_labels = pd.read_csv(Path('data/image_labels.csv'))
df_labels

,filename,target
0,0_image_1.jpg,8.0
1,0_image_2.jpg,8.0
2,0_image_3.jpg,8.0
3,0_image_4.jpg,8.0
4,1_image_1.jpg,9.8
...,...,...
1575,791_image_1.jpg,12.0
1576,792_image_1.jpg,10.5
1577,793_image_1.jpg,15.0
1578,794_image_1.jpg,12.0


In [3]:

train_df = pd.read_csv("data/train_set.csv")
valid_df = pd.read_csv("data/valid_set.csv")

train_label_dict = dict(zip(train_df['filename'], train_df['target']))
valid_label_dict = dict(zip(valid_df['filename'], valid_df['target']))
all_label_dict = {**train_label_dict, **valid_label_dict}

In [4]:
print(f"DEBUG: Train labels: {len(train_label_dict)} | Valid labels: {len(valid_label_dict)}")

# Get all image files
path = Path('images')
print(f"DEBUG: Image path set to: {path}")

all_image_files = get_image_files(path)
print(f"DEBUG: Total image files found by get_image_files: {len(all_image_files)}")

# Filter image files to only those with matching labels
processable_image_files = [f for f in all_image_files if f.name in all_label_dict]
print(f"DEBUG: Processable image files (with matching labels): {len(processable_image_files)}")

# Safety check
if len(processable_image_files) == 0:
    print("CERROR: No processable image files found (no images match labels or vice-versa).")
    if all_image_files and all_label_dict:
        print(f"  Sample image file: {all_image_files[0].name}")
        print(f"  Sample label key: {next(iter(all_label_dict.keys()))}")
        if all_image_files[0].name not in all_label_dict and all_image_files[0].name.split('.')[0] in [k.split('.')[0] for k in all_label_dict.keys()]:
            print(" Filename extensions might differ between image files and label keys.")
    raise ValueError("Cannot create DataLoaders: No matching image files and labels.")

# Helper function to get label
def get_y_func(fn):
    key = fn.name
    if key not in all_label_dict:
        print(f"DEBUG ERROR: Label not found for: {key} during get_y_func call. This should not happen if pre-filtered.")
        raise ValueError(f"Label not found for: {key}")
    return all_label_dict[key]

# Generate index lists for DataBlock IndexSplitter
filename_to_index = {f.name: i for i, f in enumerate(processable_image_files)}
valid_idxs = [filename_to_index[fname] for fname in valid_df['filename'] if fname in filename_to_index]
splitter = IndexSplitter(valid_idxs)

def convert_to_rgb(img):
    return img.convert('RGB')

# Use your existing setup for labels, image files, get_y_func, splitter, etc.

def get_datablock_variant(variant:int):
    """
    Returns a DataBlock with different augmentations depending on variant 1, 2, or 3.
    """
    # Base parameters from your original
    get_items_func = lambda _: processable_image_files
    splitter_func = splitter
    get_y_func_func = get_y_func
    item_tfms_base = RandomResizedCrop(224)

    # Different batch transforms per variant
    if variant == 1:
        batch_tfms_variant = aug_transforms(
            do_flip=False,
            max_rotate=2,
            max_zoom=1.05,
            max_lighting=0.1,
            max_warp=0.,
            p_affine=0.3,
            p_lighting=0.3
        )
    elif variant == 2:
        batch_tfms_variant = aug_transforms(
            do_flip=True,
            max_rotate=10,
            max_zoom=1.2,
            max_lighting=0.2,
            max_warp=0.2,
            p_affine=0.5,
            p_lighting=0.5
        )
    elif variant == 3:
        batch_tfms_variant = aug_transforms(
            do_flip=True,
            max_rotate=20,
            max_zoom=1.3,
            max_lighting=0.3,
            max_warp=0.3,
            p_affine=0.7,
            p_lighting=0.7
        )
    else:
        raise ValueError("variant must be 1, 2, or 3")

    return DataBlock(
        blocks=(ImageBlock, RegressionBlock),
        get_items=get_items_func,
        splitter=splitter_func,
        get_y=get_y_func_func,
        item_tfms=item_tfms_base,
        batch_tfms=batch_tfms_variant,
        n_inp=1
    )


DEBUG: Train labels: 1264 | Valid labels: 316
DEBUG: Image path set to: images
DEBUG: Total image files found by get_image_files: 2259
DEBUG: Processable image files (with matching labels): 1575


In [5]:
dls1 = get_datablock_variant(1).dataloaders(path, bs=16)
dls2 = get_datablock_variant(2).dataloaders(path, bs=16)
dls3 = get_datablock_variant(3).dataloaders(path, bs=16)

dls_list = [dls1, dls2, dls3]

Train it!

In [6]:
import torch
import torch.nn as nn

class HuberLoss(nn.Module):
    def __init__(self, delta=1.0):
        super().__init__()
        self.delta = delta

    def forward(self, input, target):
        abs_error = torch.abs(input - target)
        quadratic = torch.minimum(abs_error, torch.tensor(self.delta))
        linear = abs_error - quadratic
        loss = 0.5 * quadratic**2 + self.delta * linear
        return loss.mean()

def mae(preds, targs):
    # Ensure target shape matches preds
    if targs.ndim == 1:
        targs = targs.unsqueeze(1)
    return nn.L1Loss()(preds, targs)

class RMCELoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input, target):
        error_cubed = torch.abs(input - target) ** 3
        mean_cubed_error = torch.mean(error_cubed)
        loss = torch.pow(mean_cubed_error, 1/3)
        return loss  # must be positive

In [ ]:
import timm
from fastai.vision.all import *

model_name='efficientnet_b3'

# Create model with correct input size
model = timm.create_model(model_name, pretrained=True)

# Replace the classification head with a regression head
model.classifier = nn.Linear(model.classifier.in_features, 1)

# Create learner with the pre-configured model
learn = Learner(get_datablock_variant(1), model, metrics=[rmse, mae])

In [ ]:
#learn.fine_tune(1)

In [ ]:
#learn.recorder.values[-1]

##### Now we have a working model! For example, it predicts this picture at 13% bodyfat (not so far off in my opinion)

In [ ]:
#bf,_,probs = learn.predict(PILImage.create('images/248_image_2.jpg'))
#print(f"Bodyfat prediction: {probs[0]:.4f}")

Finally, export the model

In [ ]:
folder_tag=""
path = f"model/{model_name}{folder_tag}/model.pkl"
#learn.export(path)

In [8]:
def fine_tune_models(models=None, epochs=3, loss_func=None):

    metrics=[mae, rmse]
    """
    Creates, fine-tunes, and returns multiple timm learners for regression.

    Args:
        dls: fastai DataLoaders
        models: list of timm model names (default below)
        epochs: number of epochs to fine-tune each model
        lr: learning rate for fine_tune
        loss_func: loss function (default MSELossFlat)
        metrics: list of metrics (default mae)

    Returns:
        dict of {model_name: learner}
    """
    if models is None:
        models = [
            'resnet50d',                     # Variant of ResNet50 with minor improvements
            'mobilenetv3_large_100',         # Good performer (MobileNet family)
            'densenet121',                   # Dense connections, distinct feature learning
            'convnext_tiny',                 # Modern pure CNN with Transformer influences
            'convnext_nano',                 # Smaller, efficient ConvNeXt
            'regnety_040',                   # Systematically designed CNN (RegNet family)
            'coatnet_0_224',                 # Hybrid CNN-Transformer
            'ghostnet_100',                  # Efficient, mobile-friendly CNN
            'vit_base_patch16_224',          # Core Vision Transformer
            'deit_base_patch16_224',         # Distillation-enhanced ViT
            'swin_tiny_patch4_window7_224',  # Hierarchical Transformer, multi-scale
            'crossvit_tiny_240',             # Cross-attention, different attention mechanism
            'maxvit_tiny_rw_224',            # Newer hybrid, effective convolution-attention mix
            'hiera_tiny_224',                # Recent, state-of-the-art hierarchical Transformer
            'mambaout_tiny',                 # Novel architecture based on Mamba, distinct modeling
            'mobileone_s1',                  # Designed for efficient inference, potentially robust features
            'efficientformerv2_s0'           # Very efficient and modern hybrid architecture
        ]
    
    if loss_func is None:
        loss_func = MSELossFlat()
    if metrics is None:
        metrics = [mae]

    for model_name in models:
        db_variant = random.randint(1, 3)
        chosen_db = get_datablock_variant(db_variant)
        chosen_dls = chosen_db.dataloaders(path, bs=16)
        
        print(f"Setting up and fine-tuning {model_name}...")
        print(f"Datablock Variant: {db_variant}...")
        
        try:
            model = timm.create_model(model_name, pretrained=True)
    
            if hasattr(model, 'fc'):
                model.fc = nn.Linear(model.fc.in_features, 1)
            elif hasattr(model, 'classifier'):
                model.classifier = nn.Linear(model.classifier.in_features, 1)
            elif hasattr(model, 'head'):
                model.head = nn.Linear(model.head.in_features, 1)
            else:
                raise NotImplementedError(f"Head replacement not implemented for {model_name}")
    
            learn = Learner(chosen_dls, model, metrics=metrics)
    
            # fine_tune with given epochs and lr

            learn.fine_tune(epochs=1)
        except Exception as e:
            print (f"error in training: {e}")
            continue
    learners = {}

    for model_name in models:

        db_variant = random.randint(1, 3)
        chosen_db = get_datablock_variant(db_variant)
        chosen_dls = chosen_db.dataloaders(path, bs=16)
        
        print(f"Setting up and fine-tuning {model_name}...")
        print(f"Datablock Variant: {db_variant}...")
        
        try:
            model = timm.create_model(model_name, pretrained=True, num_classes=1)
    
            if hasattr(model, 'fc'):
                model.fc = nn.Linear(model.fc.in_features, 1)
            elif hasattr(model, 'classifier'):
                model.classifier = nn.Linear(model.classifier.in_features, 1)
            elif hasattr(model, 'head'):
                model.head = nn.Linear(model.head.in_features, 1)
            else:
                raise NotImplementedError(f"Head replacement not implemented for {model_name}")
    
            learn = Learner(chosen_dls, model, metrics=metrics)
    
            # fine_tune with given epochs and lr

            learn.fine_tune(epochs)
        except Exception as e:
            print (f"error in training: {e}")
            continue

        learners[model_name] = learn

    return learners

In [ ]:
# Assuming dls is your DataLoaders with regression targets
learners = fine_tune_models(epochs=12)

Setting up and fine-tuning resnet50d...
Datablock Variant: 1...


epoch,train_loss,valid_loss,mae,_rmse,time
0,63.135380,46.226467,5.242218,6.799005,00:52


epoch,train_loss,valid_loss,mae,_rmse,time
0,16.553440,14.571136,2.881702,3.817216,00:53


Setting up and fine-tuning mobilenetv3_large_100...
Datablock Variant: 2...


epoch,train_loss,valid_loss,mae,_rmse,time
0,40.444336,28.449968,4.154487,5.333851,00:30


...

epoch,train_loss,valid_loss,mae,_rmse,time
0,24.132814,20.063993,3.613385,4.479285,02:06


epoch,train_loss,valid_loss,mae,_rmse,time
0,19.848063,18.336012,3.343355,4.282057,02:04


Setting up and fine-tuning deit_base_patch16_224...
Datablock Variant: 3...


epoch,train_loss,valid_loss,mae,_rmse,time
0,23.220379,20.552029,3.680681,4.533434,02:12


epoch,train_loss,valid_loss,mae,_rmse,time


In [14]:
learners

4

In [12]:
import os

# learners: dict of {model_name: Learner}
for model_name, learn in learners.items():
    folder_tag = ""
    folder_path = f"model/{model_name}{folder_tag}"
    os.makedirs(folder_path, exist_ok=True)

    # Save the model
    export_path = os.path.join(folder_path, "model.pkl")
    print(f"Saving {model_name} learner to {export_path}")
    learn.export(export_path)

    # Get final metrics from recorder
    # learn.recorder.values is a list of lists: one per epoch, each containing [train_loss, valid_loss, metric1, metric2, ...]
    # learn.recorder.metric_names is the header for these values (first is 'train_loss', second 'valid_loss', etc.)
    final_epoch_metrics = learn.recorder.values[-1]  # last epoch
    metric_names = learn.recorder.metric_names[1:-1]  # skip 'epoch' at start and empty string at end

    # Build metrics string
    metrics_str = "Final epoch metrics:\n"
    for name, val in zip(metric_names, final_epoch_metrics[1:]):  # skip train_loss if you want
        metrics_str += f"{name}: {val:.6f}\n"

    # Write to metrics.txt
    metrics_path = os.path.join(folder_path, "metrics.txt")
    with open(metrics_path, "w") as f:
        f.write(metrics_str)


Saving resnet50 learner to model/resnet50/model.pkl
Saving efficientnet_b3 learner to model/efficientnet_b3/model.pkl
Saving densenet121 learner to model/densenet121/model.pkl
Saving resnet34 learner to model/resnet34/model.pkl
Saving mobilenetv3_large_100 learner to model/mobilenetv3_large_100/model.pkl
Saving efficientnet_b0 learner to model/efficientnet_b0/model.pkl
